In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# SUUMO URL（港区、渋谷区、品川区、目黒区）
url_template = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&tc=0400301&tc=0400501&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=20.0&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&et=9999999&mb=0&mt=9999999&cn=9999999&ta=13&sc=13103&sc=13113&sc=13109&sc=13110"

all_data = []
max_page = 100 

for page in range(1, max_page + 1):
    url = url_template.format(page)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    items = soup.findAll("div", {"class": "cassetteitem"})
    for item in items:
        property_name = item.find("div", {"class": "cassetteitem_content-title"}).getText().strip()
        tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")
        for tbody in tbodys:
            data = {}
            data["物件名"] = property_name
            data["階数"] = tbody.findAll("td")[2].getText().strip()
            data["家賃"] = tbody.findAll("td")[3].findAll("li")[0].getText().strip()
            data["間取り"] = tbody.findAll("td")[5].findAll("li")[0].getText().strip()
            data["面積"] = tbody.findAll("td")[5].findAll("li")[1].getText().strip()
            all_data.append(data)

# DataFrameに変換
df = pd.DataFrame(all_data)

# 重複データの除外
df_unique = df.drop_duplicates(subset=['階数', '家賃', '間取り', '面積'])

# 結果を表示
print(df_unique.head())


             物件名  階数      家賃  間取り       面積
0            平林荘  1階    13万円  4DK  70.49m2
1  kinomori代々木上原  4階  12.7万円   2K   23.7m2
2  kinomori代々木上原  1階    13万円   2K  24.08m2
3  kinomori代々木上原  2階  13.1万円   2K  24.08m2
4  kinomori代々木上原  3階  13.2万円   2K  24.08m2
一意の物件数: 52


In [57]:
# 設定
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('green-torus-405215-98138f22cf95.json', scope)
gc = gspread.authorize(credentials)

# スプレッドシートを開く、もしくは新規作成
try:
    spreadsheet = gc.open('Tech0 1130 Homework')
except gspread.SpreadsheetNotFound:
    spreadsheet = gc.create('Tech0 1130 Homework')
    # 必要に応じてサービスアカウントに共有設定

# 最初のワークシートを取得
worksheet = spreadsheet.get_worksheet(0)

# データをスプレッドシートに転記
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

/var/folders/ht/sdt7w_3s55d7vk724_f_w8w40000gn/T/ipykernel_23789/1536202176.py:17: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1wTUn5QnbbowUn6U1fCgwxVtXXYFejyKglNRifkMUtik',
 'updatedRange': "'シート1'!A1:E7801",
 'updatedRows': 7801,
 'updatedColumns': 5,
 'updatedCells': 39005}